In [1]:
# Include Libraries
import numpy as np
import tensorflow as tf
import gym
import tensorflow.contrib.layers as layers
from mujoco_py import load_model_from_xml, MjSim, MjViewer
import policies
import value_functions as vfuncs
import utils_pg as utils

# Environment setup
#env = "CartPole-v0"
#env="InvertedPendulum-v2"
env = "Humanoid-v2"

# discrete = isinstance(env.action_space, gym.spaces.Discrete)
# observation_dim = env.observation_space.shape[0]
# action_dim = env.action_space.n if discrete else env.action_space.shape[0]
max_ep_len = 1000
num_traj = 30
#traj_length = max_ep_len*(observation_dim + 2)
latent_size = 150
use_baseline = True



/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
# Feed forward network (multi-layer-perceptron, or mlp)

def build_mlp(mlp_input,output_size,scope,n_layers,size,output_activation=None):
    '''
    Build a feed forward network
    '''
    Input = mlp_input
    with tf.variable_scope(scope):
        # Dense Layers
        for i in range(n_layers-1):
            dense = tf.layers.dense(inputs = Input, units = size, activation = tf.nn.relu, bias_initializer=tf.constant_initializer(1.0))
            Input = dense
        # Fully Connected Layer
        out = layers.fully_connected(inputs = Input, num_outputs = output_size, activation_fn=output_activation)
    return out

In [5]:
class MetaLearner():
    def __init__(self, env, max_ep_len, num_traj,latent_size ):
        self.env = gym.make(env)
        self.discrete = isinstance(self.env.action_space, gym.spaces.Discrete)
        self.observation_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.n if self.discrete else self.env.action_space.shape[0]
        self.max_ep_len = max_ep_len
        self.num_traj = num_traj
        self.traj_length = self.max_ep_len*(self.observation_dim + 2) # TO Change
        self.use_baseline = True
        self.latent_size = latent_size
        self.feature_size = self.observation_dim + 1 + self.env.action_space.shape[0] # HC
        self.lr = 3e-3
        self.num_layers = 2
        self.layers_size = 32
        self.num_mdps = 10
        self.model = self.env.env.model
        self.sim = MjSim(self.model)
        self.gamma = 0.97
        self.decoder_len = 32

        # build model
        self.ConstructGraph()
    
    def add_placeholders(self):
        self.observation_placeholder_explore = tf.placeholder(tf.float32, shape=(None,self.observation_dim))
        if(self.discrete):
            self.action_placeholder_explore = tf.placeholder(tf.int32, shape=(None))
            self.action_placeholder_exploit = tf.placeholder(tf.int32, shape=(None))
        else:
            self.action_placeholder_explore = tf.placeholder(tf.float32, shape=(None,self.action_dim))
            self.action_placeholder_exploit= tf.placeholder(tf.float32, shape=(None,self.action_dim))

        self.baseline_target_placeholder = tf.placeholder(tf.float32, shape= None)
        self.advantage_placeholder_explore = tf.placeholder(tf.float32, shape=(None))
        
        #self.encoder_input_placeholder = tf.placeholder(tf.float32, shape= (self.num_traj,self.traj_length))
        self.encoder_input_placeholder = tf.placeholder(tf.float32, [None, None, self.feature_size])
        self.decoder_input_placeholder = tf.placeholder(tf.float32, shape= (1,self.latent_size))
        self.sequence_length_placeholder = tf.placeholder(tf.int32, [None, ])
        
        self.observation_placeholder_exploit = tf.placeholder(tf.float32, shape=(None,self.observation_dim))
        #TODO
        self.advantage_placeholder_exploit = tf.placeholder(tf.float32, shape=(None))
        
        
    def build_policy_explore(self, scope = "policy_explore"):
        if (self.discrete):
            self.action_logits = build_mlp(self.observation_placeholder_explore,self.action_dim,scope = scope,n_layers=self.num_layers,size = self.layers_size,output_activation=None)
            self.explore_action = tf.multinomial(self.action_logits,1)
            self.explore_action = tf.squeeze(self.explore_action, axis=1)
            self.explore_logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.action_logits, labels = self.action_placeholder_explore)

        else:   
            action_means = build_mlp(self.observation_placeholder_explore,self.action_dim,scope,n_layers=self.num_layers, size = self.layers_size,output_activation=None)
            init = tf.constant(np.random.rand(1, 2))
            #log_std = tf.get_variable("log_std", [self.action_dim])
            log_std = tf.get_variable(
                'log_std',
                shape=[1, self.action_dim],
                dtype=tf.float32,
                initializer=tf.zeros_initializer(),
                trainable=True
            )
            log_std = tf.log(tf.exp(log_std) + 1.)
            log_std = tf.tile(log_std,
                                   [tf.shape(action_means)[0], 1])
            self.explore_action = tf.random_normal((1,),
                                                   action_means,
                                                   log_std)
            dist = tf.contrib.distributions.MultivariateNormalDiag(action_means, log_std)
            self.explore_logprob = dist.prob(self.action_placeholder_explore, name = 'log_prob')
            #self.explore_action =   action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
            #mvn = tf.contrib.distributions.MultivariateNormalDiag(action_means, tf.exp(log_std))
            #self.explore_logprob =  mvn.log_prob(value = self.action_placeholder_explore, name='log_prob')

    
    
    def build_policy_exploit(self, scope = "policy_exploit"):
        if(self.discrete):
            #self.exploit_action_logits = (tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W1) + self.d_B1), self.d_W2) + self.d_B2),self.d_W3) + self.d_B3)
            #self.exploit_action_logits = tf.matmul(self.observation_placeholder_exploit,self.d_W3) + self.d_B3
            self.exploit_action_logits = tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W2) + self.d_B2), self.d_W3) + self.d_B3
            self.exploit_action = tf.multinomial(self.exploit_action_logits,1)
            self.exploit_action = tf.squeeze(self.exploit_action, axis=1)
            self.exploit_logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.exploit_action_logits, labels = self.action_placeholder_exploit)
        else:
            action_means = tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W2) + self.d_B2), self.d_W3) + self.d_B3
            init = tf.constant(np.random.rand(1, 2))
            log_std = tf.get_variable("exploit_log_prob", [self.action_dim])
            self.exploit_action = action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
            self.exploit_logprob = utils.gauss_log_prob(mu=action_means, logstd=log_std, x=self.action_placeholder_exploit)
            
#             init = tf.constant(np.random.rand(1, 2))
#             log_std = tf.get_variable("exploit_log_prob", [self.action_dim])
#             self.exploit_action =   action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
#             mvn = tf.contrib.distributions.MultivariateNormalDiag(action_means, tf.exp(log_std))
#             self.exploit_logprob =  mvn.log_prob(value = self.action_placeholder_exploit, name='exploit_log_prob')

        #self.loss_grads_exploit = self.exploit_logprob * self.advantage_placeholder_exploit
        
    def NNEncoder(self, scope = "NNEncoder"):
        self.Z = build_mlp(self.encoder_input_placeholder,self.latent_size,scope = scope,n_layers=3,size = 60,output_activation=None)
    
 

    # input [num_traj, length, features (obs + action + reward) ]
    def LSTMEncoder(self, scope = "LSTMEncoder"):
        self.hidden_size = 64
        initializer = tf.random_uniform_initializer(-1, 1)
        cell = tf.nn.rnn_cell.LSTMCell(self.hidden_size, self.feature_size, initializer=initializer)
        cell_out = tf.contrib.rnn.OutputProjectionWrapper(cell, self.latent_size)
        self.output, _ = tf.nn.dynamic_rnn(cell_out,self.encoder_input_placeholder,self.sequence_length_placeholder,dtype=tf.float32,)
        batch_size = tf.shape(self.output)[0]
        max_length = tf.shape(self.output)[1]
        out_size = int(self.output.get_shape()[2])
        index = tf.range(0, batch_size) * max_length + (self.sequence_length_placeholder - 1)
        flat = tf.reshape(self.output, [-1, out_size])
        self.Z = tf.reduce_mean(tf.gather(flat, index), axis = 0)
        
    
    def Decoder(self, decoder_out_dim, scope):
        return build_mlp(self.decoder_input_placeholder,decoder_out_dim,scope = scope,n_layers=1,size = 64,output_activation=None)
    
    
    def sample_paths_explore(self, env,mdp_num,Test = False, num_episodes = None):
        paths = []
        self.length = []
        episode_rewards = []
        self.sim.model.opt.gravity[-1] = -5 - mdp_num
        for i in range(self.num_traj):
            pad = False
            state = env.reset()
            new_states,states,new_actions, actions, rewards,new_rewards = [], [], [], [], [],[]
            episode_reward = 0
            for step in range(self.max_ep_len):
                if (pad):
                    states.append([0]*self.observation_dim)
                    if(self.discrete):
                        actions.append(0)
                    else:
                        actions.append([0]*self.action_dim)
                    rewards.append(0)
                else:
                    states.append(state)
                    new_states.append(state)
                    #action = self.sess.run(self.explore_action, feed_dict={self.observation_placeholder_explore : states[-1][None]})[0]
                    action = self.policyfn.sample_action(state) #NEW
                    state, reward, done, info = env.step(action)
                    actions.append(action)
                    new_actions.append(action)
                    rewards.append(reward)
                    new_rewards.append(reward)
                    episode_reward += reward
                    if (done or step == self.max_ep_len-1):
                        episode_rewards.append(episode_reward)
                        self.length.append(step + 1)
                        pad = True 
                        
            episode_rewards.append(episode_reward)            
            #print("explore",np.array(actions))
            path = {"new_obs" : np.array(new_states),"observation" : np.array(states),
                                "reward" : np.array(rewards),"new_acs" : np.array(new_actions),
                                "action" : np.array(actions),"new_rewards":np.array(new_rewards)}
            paths.append(path)
        return paths, episode_rewards
    
    def sample_paths_exploit(self, env,Z,mdp_num,Test = False, num_episodes = None):
        self.sim.model.opt.gravity[-1] = -5 - mdp_num
        
        paths = []
        num = 0
        episode_rewards = []
        for i in range(self.num_traj):
            state = env.reset()
            states, actions, rewards = [], [], []
            episode_reward = 0
            for step in range(self.max_ep_len):
                states.append(state)
                action = self.sess.run(self.exploit_action, feed_dict={self.observation_placeholder_exploit : state[None], self.decoder_input_placeholder: Z})[0]
                state, reward, done, info = env.step(action)
                actions.append(action)
                rewards.append(reward)
                episode_reward += reward
                if (done or step == self.max_ep_len-1):
                    episode_rewards.append(episode_reward)
                    break
            #print("exploit",np.array(actions))
            path = {"observation" : np.array(states),
                                "reward" : np.array(rewards),
                                "action" : np.array(actions)}
            paths.append(path)
 
        #print("exploit success: ", num)
        return paths, episode_rewards
    
    def get_returns(self,paths, explore = False):
        all_returns = []
        m = 0
        for path in paths:
            rewards = path["reward"]
            returns = []
            if(explore):
                length = self.length[m]
            else:
                length = len(rewards)
            for i in range(length):
                path_returns = 0
                k = 0
                for j in range(i,length):
                    path_returns = path_returns + rewards[j]*(self.gamma)**k
                    k = k+1
                returns.append(path_returns)
            all_returns.append(returns)
            m+=1
        returns = np.concatenate(all_returns)
        return returns
    
    def stack_trajectories(self,paths):
        trajectories = []
        for path in paths:
            rewards = path["reward"]
            states = path["observation"]
            action = path["action"]
            
            SAR = []
            for i in range(len(states)):
                SAR.append(list(states[i]) + list(action[i]) + [rewards[i]])
            trajectories.append(SAR)
        return np.array(trajectories)
    
    def addBaseline(self):
        self.baseline = build_mlp(self.observation_placeholder_explore,1,scope = "baseline",n_layers=self.num_layers, size = self.layers_size,output_activation=None)
        self.baseline_loss = tf.losses.mean_squared_error(self.baseline_target_placeholder,self.baseline,scope = "baseline")
        baseline_adam_optimizer =  tf.train.AdamOptimizer(learning_rate = self.lr)
        self.update_baseline_op = baseline_adam_optimizer.minimize(self.baseline_loss)

    def calculate_advantage(self,returns, observations):
        if (self.use_baseline):
            baseline = self.sess.run(self.baseline, {input_placeholder:observations})
            adv = returns - baseline
            adv = (adv - np.mean(adv))/np.std(adv)
        else:
            adv = returns
        return adv
    

        
    def ConstructGraph(self):
        tf.reset_default_graph()
        self.sess = tf.Session()
        
        self.vf = vfuncs.NnValueFunction(session=self.sess,ob_dim=self.observation_dim)
        self.policyfn = policies.GaussianPolicy(self.sess,self.observation_dim, self.action_dim)
        
        self.add_placeholders()
        
#         self.add_placeholders()
        
#         self.build_policy_explore()
#         self.explore_policy_loss = -tf.reduce_sum(self.explore_logprob * self.advantage_placeholder_explore)
#         self.loss_grads_explore = -self.explore_logprob * self.advantage_placeholder_explore
#         self.tvars_explore = tf.trainable_variables()
#         self.gradients_explore = tf.gradients(self.explore_policy_loss,self.tvars_explore)
        
#         #self.addBaseline()
        
#         self.baseline = build_mlp(self.observation_placeholder_explore,1,scope = "baseline",n_layers=1, size = 16,output_activation=None)
#         self.baseline_loss = tf.losses.mean_squared_error(self.baseline_target_placeholder,self.baseline,scope = "baseline")
#         baseline_adam_optimizer =  tf.train.AdamOptimizer(learning_rate = self.lr)
#         self.update_baseline_op = baseline_adam_optimizer.minimize(self.baseline_loss)
        
#         #Encoder LSTM
        self.LSTMEncoder()
        
        
        #decoder weights
        #self.d_W1 = self.Decoder(scope = "W1", decoder_out_dim = self.observation_dim*self.decoder_len)
        self.d_W2 = self.Decoder(scope = "W2", decoder_out_dim = self.observation_dim*self.decoder_len)
        #self.d_W3 = self.Decoder(scope = "W3", decoder_out_dim = self.decoder_len*action_dim)
        self.d_W3 = self.Decoder(scope = "W3", decoder_out_dim = self.decoder_len*self.action_dim)
        
        #self.d_W1 = ((self.d_W1 - (tf.reduce_max(self.d_W1) + tf.reduce_min(self.d_W1))/2)/(tf.reduce_max(self.d_W1) - tf.reduce_min(self.d_W1)))*2 
        #self.d_W2 = ((self.d_W2 - (tf.reduce_max(self.d_W2) + tf.reduce_min(self.d_W2))/2)/(tf.reduce_max(self.d_W2) - tf.reduce_min(self.d_W2)))*2 
        #self.d_W3 = ((self.d_W3 - (tf.reduce_max(self.d_W3) + tf.reduce_min(self.d_W3))/2)/(tf.reduce_max(self.d_W3) - tf.reduce_min(self.d_W3)))*2 
        
        #self.d_W1 = tf.reshape(self.d_W1, [self.observation_dim, self.decoder_len])
        self.d_W2 = tf.reshape(self.d_W2, [self.observation_dim, self.decoder_len])
        #self.d_W3 = tf.reshape(self.d_W3, [self.decoder_len, self.action_dim])
        self.d_W3 = tf.reshape(self.d_W3, [self.decoder_len, self.action_dim])
        
        # decoder output bias
        #self.d_B1 = tf.reshape(self.Decoder(decoder_out_dim = self.decoder_len, scope = "B1"), [self.decoder_len])
        self.d_B2 = tf.reshape(self.Decoder(decoder_out_dim = self.decoder_len, scope = "B2"), [self.decoder_len])
        self.d_B3 = tf.reshape(self.Decoder(decoder_out_dim = self.action_dim, scope = "B3"), [self.action_dim])
        #self.d_B1 = ((self.d_B1 - (tf.reduce_max(self.d_B1) + tf.reduce_min(self.d_B1))/2)/(tf.reduce_max(self.d_B1) - tf.reduce_min(self.d_B1)))*2 
        #self.d_B2 = ((self.d_B2 - (tf.reduce_max(self.d_B2) + tf.reduce_min(self.d_B2))/2)/(tf.reduce_max(self.d_B2) - tf.reduce_min(self.d_B2)))*2 
        
        
        # exploit policy
        self.build_policy_exploit()
        #self.d = [self.d_W1, self.d_B1, self.d_W2, self.d_B2, self.d_W3, self.d_B3]
        self.exploit_policy_loss = -tf.reduce_sum(self.exploit_logprob * self.advantage_placeholder_exploit)
        self.d = [self.d_W3, self.d_B3]
        self.gradients_exploit = tf.gradients(self.exploit_policy_loss,self.d)
        
        # train encoder and decoder
        adam_optimizer_exploit =  tf.train.AdamOptimizer(self.lr*0.1)
        self.output_train_op = adam_optimizer_exploit.minimize(self.exploit_policy_loss)
        # train original network
#         adam_optimizer_explore = tf.train.AdamOptimizer(self.lr)
#         self.input_train_op = adam_optimizer_explore.minimize(self.explore_policy_loss)
        init = tf.global_variables_initializer()
        self.sess.run(init)
    
    def initialize(self):
        self.ConstructGraph()
#         # create tf session
#         self.sess = tf.Session()
        
#         # initiliaze all variables
#         init = tf.global_variables_initializer()
#         self.sess.run(init)
    
    def train_step(self): 
        # sample num_traj*num_MDPs
        for i in range(self.num_mdps):
            explore_paths, explore_rewards = self.sample_paths_explore(self.env,i)
            observations_explore = np.concatenate([path["observation"] for path in explore_paths])
            new_observations_explore = np.concatenate([path["new_obs"] for path in explore_paths])
            new_actions_explore = np.concatenate([path["new_acs"] for path in explore_paths])
            actions_explore = np.concatenate([path["action"] for path in explore_paths])
            rewards_explore = np.concatenate([path["reward"] for path in explore_paths])
            
            vtargs, vpreds, advantages_explore = [], [], []
            for path in explore_paths:
                rew_t = path["new_rewards"]
                return_t = utils.discount(rew_t, self.gamma)
                vpred_t = self.vf.predict(path["new_obs"])
                adv_t = return_t - vpred_t
                advantages_explore.append(adv_t)
                vtargs.append(return_t)
                vpreds.append(vpred_t)
                
                
            #returns_explore = self.get_returns(explore_paths, explore = True)
            advantages_explore = np.concatenate(advantages_explore)
            advantages_explore = (advantages_explore - advantages_explore.mean()) / (advantages_explore.std() + 1e-8)
            vtarg_n = np.concatenate(vtargs)
            vpred_n = np.concatenate(vpreds)
            #update baseline
            #print(advantages_explore.shape,new_observations_explore.shape)
            self.vf.fit(new_observations_explore, vtarg_n)
            
            surr_loss, oldmean_na, oldlogstd_a = self.policyfn.update_policy(
                    new_observations_explore, new_actions_explore, advantages_explore, self.lr)
            
            print("average reward explore:", np.mean(explore_rewards))
            
            #print(returns_explore)
#             print("average reward explore: MDP", i, np.sum(explore_rewards)/num_traj, len(explore_rewards))

#             baseline_explore = self.sess.run(self.baseline, {self.observation_placeholder_explore:new_observations_explore})
#             adv = returns_explore - np.squeeze(baseline_explore)
#             advantages_explore = (adv - np.mean(adv))/(np.std(adv) + 1e-12)


#             # update the baseline

#             self.sess.run(self.update_baseline_op, {self.observation_placeholder_explore:new_observations_explore, 
#                                            self.baseline_target_placeholder : returns_explore})

            # calculate explore gradients
    #         grads_explore = self.sess.run(self.gradients_explore, feed_dict={
    #                     self.observation_placeholder_explore : observations_explore,
    #                     self.action_placeholder_explore : actions_explore,
    #                     self.advantage_placeholder_explore : returns_explore})
            #print("explore",grads_explore )
            # form trajectory matrix

            M = np.array(self.stack_trajectories(explore_paths))


            #encoder LSTM
            Z = self.sess.run(self.Z, feed_dict = {self.encoder_input_placeholder: M,self.sequence_length_placeholder: self.length })
            Z = np.reshape(Z,[1,len(Z)])
#             #print("Z",Z)
#             #print(self.sess.run(self.d, feed_dict = {self.decoder_input_placeholder: Z}))
#             #print("d",self.d)
#             # sample paths
#             tvars = tf.trainable_variables()
#             tvars_vals = self.sess.run(tvars[-5:-1])
#             #print(tvars_vals)
            for j in range(10):
                exploit_paths, exploit_rewards = self.sample_paths_exploit(self.env,Z,i)
                # get observations, actions and rewards
                observations_exploit = np.concatenate([path["observation"] for path in exploit_paths])
                actions_exploit = np.concatenate([path["action"] for path in exploit_paths])
                rewards_exploit = np.concatenate([path["reward"] for path in exploit_paths])
                returns_exploit = self.get_returns(exploit_paths)
                print("average reward exploit: MDP", i, np.sum(exploit_rewards) / num_traj, len(exploit_rewards))


#             # exploit grads
#     #         grads_exploit = self.sess.run(self.gradients_exploit,feed_dict={
#     #                     self.observation_placeholder_exploit : observations_exploit,
#     #                     self.action_placeholder_exploit : actions_exploit,
#     #                     self.advantage_placeholder_exploit : returns_exploit,
#     #                     self.decoder_input_placeholder: Z})

            #train encoder and decoder network
                self.sess.run(self.output_train_op, feed_dict={
                                self.observation_placeholder_exploit : observations_exploit,
                                self.action_placeholder_exploit : actions_exploit,
                                self.advantage_placeholder_exploit : returns_exploit,
                                self.decoder_input_placeholder: Z})

            # find advantage for input network
    #         advantage_explore = 0
    #         for i in range(len(grads_exploit)):
    #             l1 = grads_exploit[i]
    #             l2 = grads_explore[i]
    #             advantage_explore = advantage_explore + np.matmul(l1.flatten(), l2.flatten())

            # train input policy
#             self.sess.run(self.input_train_op, feed_dict={
#                             self.observation_placeholder_explore : new_observations_explore,
#                             self.action_placeholder_explore : new_actions_explore,
#                             self.advantage_placeholder_explore : advantages_explore})
    
    def test(self):
        explore_paths, explore_rewards = self.sample_paths_explore(self.env,20*np.random.rand())
        print("gravity: ",self.sim.model.opt.gravity )
        M = self.stack_trajectories(explore_paths)
        Z = self.sess.run(self.Z, feed_dict = {self.encoder_input_placeholder: M,self.sequence_length_placeholder: self.length })
        Z = np.reshape(Z,[1,len(Z)])
        # sample paths
        exploit_paths, exploit_rewards = self.sample_paths_exploit(self.env,Z,20*np.random.rand())
        print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))
        
    def train(self):
        self.initialize()
        num_epochs = 200
        for epoch in range(num_epochs):
            self.num_traj = 20
            print("epoch number: ", epoch)
            self.train_step()
            print("evaluating on new MDPs")
            self.num_traj = 400
            self.test()
            

In [6]:
a = MetaLearner(env, max_ep_len, num_traj, latent_size)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [7]:
a.train()

epoch number:  0
average reward explore: 87.89810504610844
average reward exploit: MDP 0 70.67102469406363 20
average reward exploit: MDP 0 56.9229227137672 20
average reward exploit: MDP 0 58.806632485294294 20
average reward exploit: MDP 0 59.07209559249964 20
average reward exploit: MDP 0 57.0035024271645 20
average reward exploit: MDP 0 59.071488782162845 20
average reward exploit: MDP 0 55.15726973189076 20
average reward exploit: MDP 0 58.31024338805231 20
average reward exploit: MDP 0 57.0984469601896 20
average reward exploit: MDP 0 57.79130592306911 20
average reward explore: 84.0697328782916
average reward exploit: MDP 1 49.68789335136884 20
average reward exploit: MDP 1 51.28713936863396 20
average reward exploit: MDP 1 54.95621693150508 20
average reward exploit: MDP 1 63.23139727293232 20
average reward exploit: MDP 1 76.55806374232762 20
average reward exploit: MDP 1 95.5536503244824 20
average reward exploit: MDP 1 106.09777742265584 20
average reward exploit: MDP 1 102.

average reward exploit: MDP 4 49.12162240112514 20
average reward exploit: MDP 4 48.01776439952097 20
average reward exploit: MDP 4 45.18155250799414 20
average reward exploit: MDP 4 46.272394239307445 20
average reward exploit: MDP 4 46.37933500190595 20
average reward explore: 70.06068097015786
average reward exploit: MDP 5 43.783824143093995 20
average reward exploit: MDP 5 45.66788002252704 20
average reward exploit: MDP 5 47.76322651465385 20
average reward exploit: MDP 5 48.702136445419164 20
average reward exploit: MDP 5 51.688346085902616 20
average reward exploit: MDP 5 55.024022292768194 20
average reward exploit: MDP 5 54.61702685994758 20
average reward exploit: MDP 5 56.59535561061875 20
average reward exploit: MDP 5 54.86336677396745 20
average reward exploit: MDP 5 57.2467184156978 20
average reward explore: 67.12222934957988
average reward exploit: MDP 6 54.150360664473574 20
average reward exploit: MDP 6 56.2440483361329 20
average reward exploit: MDP 6 54.615375225716

average reward exploit: MDP 9 36.653786253302755 20
average reward exploit: MDP 9 36.581127786715015 20
average reward exploit: MDP 9 36.88643455636521 20
average reward exploit: MDP 9 39.32744146544621 20
average reward exploit: MDP 9 39.494137283349325 20
average reward exploit: MDP 9 39.44467079241338 20
average reward exploit: MDP 9 39.37477077844574 20
average reward exploit: MDP 9 39.73196418430064 20
average reward exploit: MDP 9 39.54884729083745 20
evaluating on new MDPs
gravity:  [  0.           0.         -14.64366015]
average reward exploit 818.2004798669532 400
epoch number:  3
average reward explore: 86.50264549943772
average reward exploit: MDP 0 43.260424424410466 20
average reward exploit: MDP 0 43.56992190158436 20
average reward exploit: MDP 0 44.0065618950582 20
average reward exploit: MDP 0 43.91812720911541 20
average reward exploit: MDP 0 44.025388800736394 20
average reward exploit: MDP 0 43.72605237107578 20
average reward exploit: MDP 0 43.117647852729746 20
a

average reward exploit: MDP 3 40.48710428269362 20
average reward exploit: MDP 3 40.0865645800085 20
average reward exploit: MDP 3 40.23775879211912 20
average reward exploit: MDP 3 40.78991762818598 20
average reward exploit: MDP 3 41.100968090075575 20
average reward explore: 64.8034223785269
average reward exploit: MDP 4 37.731216300497145 20
average reward exploit: MDP 4 38.062828355351584 20
average reward exploit: MDP 4 37.93412419596034 20
average reward exploit: MDP 4 37.45595065653505 20
average reward exploit: MDP 4 36.99808828802312 20
average reward exploit: MDP 4 38.06206914663649 20
average reward exploit: MDP 4 37.11367833966119 20
average reward exploit: MDP 4 38.12537742934856 20
average reward exploit: MDP 4 37.69928772000527 20
average reward exploit: MDP 4 37.67222607082049 20
average reward explore: 60.34076216368021
average reward exploit: MDP 5 35.923525072083756 20
average reward exploit: MDP 5 35.93127560469137 20
average reward exploit: MDP 5 35.93729977008493

average reward exploit: MDP 8 59.283079025649315 20
average reward exploit: MDP 8 58.06182886367091 20
average reward exploit: MDP 8 58.44810590833074 20
average reward exploit: MDP 8 57.392360301978286 20
average reward exploit: MDP 8 58.652918450764524 20
average reward exploit: MDP 8 58.816836474673096 20
average reward exploit: MDP 8 58.23638416424392 20
average reward exploit: MDP 8 58.25571964971186 20
average reward exploit: MDP 8 58.24669689540618 20
average reward explore: 55.62829038892537
average reward exploit: MDP 9 54.19761823343362 20
average reward exploit: MDP 9 54.54742059814707 20
average reward exploit: MDP 9 54.49503756017104 20
average reward exploit: MDP 9 55.091478152565784 20
average reward exploit: MDP 9 54.395210300886816 20
average reward exploit: MDP 9 54.75005058319225 20
average reward exploit: MDP 9 54.25257169256479 20
average reward exploit: MDP 9 54.21095378174455 20
average reward exploit: MDP 9 54.19045113648748 20
average reward exploit: MDP 9 53.7

average reward exploit: MDP 2 42.352370361055904 20
average reward exploit: MDP 2 42.308384925779144 20
average reward exploit: MDP 2 42.13337466369767 20
average reward exploit: MDP 2 42.334588334657774 20
average reward exploit: MDP 2 42.325714031977896 20
average reward explore: 70.80389091375756
average reward exploit: MDP 3 64.12567246635498 20
average reward exploit: MDP 3 54.834265279581935 20
average reward exploit: MDP 3 60.884528605089784 20
average reward exploit: MDP 3 60.19949288409129 20
average reward exploit: MDP 3 70.25390652117467 20
average reward exploit: MDP 3 53.89997826750631 20
average reward exploit: MDP 3 45.35588408997282 20
average reward exploit: MDP 3 51.86928448000386 20
average reward exploit: MDP 3 68.09607725130515 20
average reward exploit: MDP 3 70.17380901805535 20
average reward explore: 67.05097627762137
average reward exploit: MDP 4 58.55590499394202 20
average reward exploit: MDP 4 58.317084754573834 20
average reward exploit: MDP 4 56.870638907

average reward exploit: MDP 7 33.14379985904803 20
average reward exploit: MDP 7 32.83983536854596 20
average reward exploit: MDP 7 33.29434569072367 20
average reward exploit: MDP 7 31.064730187626207 20
average reward exploit: MDP 7 31.12183038286298 20
average reward exploit: MDP 7 30.711513731647365 20
average reward exploit: MDP 7 30.250933395921667 20
average reward exploit: MDP 7 32.04818621879987 20
average reward exploit: MDP 7 31.813000697667697 20
average reward exploit: MDP 7 31.818745649737416 20
average reward explore: 53.50639936706095
average reward exploit: MDP 8 30.83214753569807 20
average reward exploit: MDP 8 30.89031851024956 20
average reward exploit: MDP 8 31.076214820903328 20
average reward exploit: MDP 8 30.808986668201644 20
average reward exploit: MDP 8 31.254451905474607 20
average reward exploit: MDP 8 30.38056059767009 20
average reward exploit: MDP 8 30.203783731846375 20
average reward exploit: MDP 8 30.957811131407833 20
average reward exploit: MDP 8 

KeyboardInterrupt: 

In [16]:
for epoch in range(200):
        print("epoch number: ", epoch)
        a.train_step()

epoch number:  0
average reward explore: 82.96251755642649
average reward exploit: MDP 0 73.82496456351859 30
average reward exploit: MDP 0 72.63798571588255 30
average reward exploit: MDP 0 72.9195911751872 30
average reward exploit: MDP 0 73.57823085200245 30
average reward exploit: MDP 0 73.94708967913637 30
average reward exploit: MDP 0 74.49656268863008 30
average reward exploit: MDP 0 73.65975738682121 30
average reward exploit: MDP 0 75.16580731982253 30
average reward exploit: MDP 0 74.54466412274041 30
average reward exploit: MDP 0 73.5883272924395 30
average reward explore: 78.37416715003496
average reward exploit: MDP 1 67.51819471570535 30
average reward exploit: MDP 1 67.97158607986732 30
average reward exploit: MDP 1 66.72792142490734 30
average reward exploit: MDP 1 74.5388548880442 30
average reward exploit: MDP 1 84.03547942829246 30
average reward exploit: MDP 1 84.68315754125462 30
average reward exploit: MDP 1 84.1231174736365 30
average reward exploit: MDP 1 80.079

average reward exploit: MDP 4 103.82656167706229 30
average reward exploit: MDP 4 106.74874124081475 30
average reward exploit: MDP 4 107.61798199112913 30
average reward explore: 55.44522103852402
average reward exploit: MDP 5 97.57707892030976 30
average reward exploit: MDP 5 97.35794309574439 30
average reward exploit: MDP 5 97.14304035226478 30
average reward exploit: MDP 5 96.75155723414983 30
average reward exploit: MDP 5 97.95902305871641 30
average reward exploit: MDP 5 98.16005921381894 30
average reward exploit: MDP 5 98.43185242857135 30
average reward exploit: MDP 5 97.15975616034534 30
average reward exploit: MDP 5 96.42401201018356 30
average reward exploit: MDP 5 98.07924893318658 30
average reward explore: 52.21646807267904
average reward exploit: MDP 6 98.82165005722794 30
average reward exploit: MDP 6 95.56435337720399 30
average reward exploit: MDP 6 95.24608281692635 30
average reward exploit: MDP 6 95.8071353368924 30
average reward exploit: MDP 6 95.54619788171735

average reward exploit: MDP 9 48.998675166760876 30
average reward exploit: MDP 9 48.97631813856061 30
average reward exploit: MDP 9 48.98356888138383 30
average reward exploit: MDP 9 49.04473449566569 30
average reward exploit: MDP 9 48.98695784681199 30
epoch number:  3
average reward explore: 81.89384749909992
average reward exploit: MDP 0 71.2346432749161 30
average reward exploit: MDP 0 70.28655539223102 30
average reward exploit: MDP 0 70.77535255736056 30
average reward exploit: MDP 0 70.5225059208699 30
average reward exploit: MDP 0 70.23771475628105 30
average reward exploit: MDP 0 68.8419965769967 30
average reward exploit: MDP 0 70.20093058808654 30
average reward exploit: MDP 0 71.57894717266548 30
average reward exploit: MDP 0 70.3250963406393 30
average reward exploit: MDP 0 71.42248928948986 30
average reward explore: 76.49678503106965
average reward exploit: MDP 1 64.30695015566421 30
average reward exploit: MDP 1 65.19956960400094 30
average reward exploit: MDP 1 65.66

average reward exploit: MDP 4 52.94277566161709 30
average reward exploit: MDP 4 52.89045764120508 30
average reward exploit: MDP 4 52.83260943217138 30
average reward exploit: MDP 4 52.451106552446625 30
average reward exploit: MDP 4 52.42036412047077 30
average reward exploit: MDP 4 51.875005268701734 30
average reward exploit: MDP 4 52.82398235024883 30
average reward explore: 63.933107610325784
average reward exploit: MDP 5 51.11134724537389 30
average reward exploit: MDP 5 51.378204326913504 30
average reward exploit: MDP 5 50.894521301115915 30
average reward exploit: MDP 5 51.55437040849117 30
average reward exploit: MDP 5 50.65754005266491 30
average reward exploit: MDP 5 50.19004309521834 30
average reward exploit: MDP 5 51.2355832265118 30
average reward exploit: MDP 5 51.22204190752637 30
average reward exploit: MDP 5 51.7334290097888 30
average reward exploit: MDP 5 50.29785398781415 30
average reward explore: 62.1257571773232
average reward exploit: MDP 6 48.06396829335037

average reward exploit: MDP 9 44.32430036725562 30
average reward exploit: MDP 9 45.261324468493164 30
average reward exploit: MDP 9 43.87523388186061 30
average reward exploit: MDP 9 44.85740674424618 30
average reward exploit: MDP 9 45.05557786045133 30
average reward exploit: MDP 9 44.67317379482956 30
average reward exploit: MDP 9 45.64198274423094 30
average reward exploit: MDP 9 44.64079683065837 30
average reward exploit: MDP 9 44.89994515779729 30
epoch number:  6
average reward explore: 83.07040747490943
average reward exploit: MDP 0 92.88432619783582 30
average reward exploit: MDP 0 93.38040797132786 30
average reward exploit: MDP 0 93.41461219111642 30
average reward exploit: MDP 0 93.99440564312984 30
average reward exploit: MDP 0 93.95803026987466 30
average reward exploit: MDP 0 87.76285248932255 30
average reward exploit: MDP 0 83.05423236633432 30
average reward exploit: MDP 0 83.81528665441506 30
average reward exploit: MDP 0 83.83455980408257 30
average reward exploit

...

average reward exploit: MDP 7 47.767914299419395 30
average reward exploit: MDP 7 47.97102641225295 30
average reward exploit: MDP 7 47.07688702058965 30
average reward exploit: MDP 7 47.951114741911525 30
average reward exploit: MDP 7 47.476636662686325 30
average reward explore: 55.00265414256167
average reward exploit: MDP 8 46.30578258542317 30
average reward exploit: MDP 8 46.85893515161296 30
average reward exploit: MDP 8 45.6452704022918 30
average reward exploit: MDP 8 47.24339839248315 30
average reward exploit: MDP 8 45.88529911542695 30
average reward exploit: MDP 8 46.51814140591761 30
average reward exploit: MDP 8 46.50982375119046 30
average reward exploit: MDP 8 46.458269334313066 30
average reward exploit: MDP 8 46.17151534423731 30
average reward exploit: MDP 8 45.84295466248245 30
average reward explore: 53.42533774087668
average reward exploit: MDP 9 44.28797485259867 30
average reward exploit: MDP 9 44.241849988746374 30
average reward exploit: MDP 9 44.263558530577

average reward exploit: MDP 2 58.03668365028167 30
average reward exploit: MDP 2 57.657014743007956 30
average reward exploit: MDP 2 58.235104610676906 30
average reward exploit: MDP 2 57.94128480839893 30
average reward exploit: MDP 2 58.16533881633333 30
average reward exploit: MDP 2 58.339335524071615 30
average reward exploit: MDP 2 58.538624913181636 30
average reward exploit: MDP 2 57.97664056854067 30
average reward explore: 70.14960539298642
average reward exploit: MDP 3 56.087535313469054 30
average reward exploit: MDP 3 54.87680716166672 30
average reward exploit: MDP 3 55.06822887586133 30
average reward exploit: MDP 3 55.93132982969053 30
average reward exploit: MDP 3 55.915955947514725 30
average reward exploit: MDP 3 55.56780893822291 30
average reward exploit: MDP 3 55.62526298102205 30
average reward exploit: MDP 3 55.84215947133806 30
average reward exploit: MDP 3 56.181353390343425 30
average reward exploit: MDP 3 56.02985343523476 30
average reward explore: 66.264852

average reward exploit: MDP 7 47.87769444091493 30
average reward exploit: MDP 7 47.743087383394204 30
average reward exploit: MDP 7 47.055073409840006 30
average reward exploit: MDP 7 48.32812757300708 30
average reward exploit: MDP 7 47.27551220675402 30
average reward exploit: MDP 7 47.528685919474455 30
average reward exploit: MDP 7 47.31793582970886 30
average reward exploit: MDP 7 47.161605420357155 30
average reward exploit: MDP 7 48.285436263529476 30
average reward exploit: MDP 7 46.994844583494796 30
average reward explore: 54.784925374336154
average reward exploit: MDP 8 46.38210059490739 30
average reward exploit: MDP 8 46.39870422558361 30
average reward exploit: MDP 8 46.71625379195284 30
average reward exploit: MDP 8 45.95005034922723 30
average reward exploit: MDP 8 46.268894105631425 30
average reward exploit: MDP 8 46.46989198236249 30
average reward exploit: MDP 8 46.616090527128414 30
average reward exploit: MDP 8 46.417588499634 30
average reward exploit: MDP 8 46.

average reward exploit: MDP 1 61.58503374082059 30
average reward exploit: MDP 1 62.68821345366691 30
average reward explore: 76.36374391574476
average reward exploit: MDP 2 58.38414406810781 30
average reward exploit: MDP 2 58.6863132705267 30
average reward exploit: MDP 2 57.935036434011096 30
average reward exploit: MDP 2 58.344031362358926 30
average reward exploit: MDP 2 58.10073635590925 30
average reward exploit: MDP 2 58.35883251341773 30
average reward exploit: MDP 2 58.55122478943885 30
average reward exploit: MDP 2 58.279555396056494 30
average reward exploit: MDP 2 58.91468284893606 30
average reward exploit: MDP 2 57.7662881146136 30
average reward explore: 72.74671283816407
average reward exploit: MDP 3 55.29519431738791 30
average reward exploit: MDP 3 56.25815903604395 30
average reward exploit: MDP 3 55.575140906932575 30
average reward exploit: MDP 3 56.2864204091456 30
average reward exploit: MDP 3 55.94624475523525 30
average reward exploit: MDP 3 56.2308275208326 3

average reward exploit: MDP 6 48.14546285732521 30
average reward exploit: MDP 6 47.40052686963064 30
average reward exploit: MDP 6 47.51361734495236 30
average reward exploit: MDP 6 47.755639739308954 30
average reward explore: 61.48038951850906
average reward exploit: MDP 7 47.72820867581065 30
average reward exploit: MDP 7 47.144622920149416 30
average reward exploit: MDP 7 47.321356901830356 30
average reward exploit: MDP 7 48.12192375480393 30
average reward exploit: MDP 7 47.73916254470809 30
average reward exploit: MDP 7 48.09216811439426 30
average reward exploit: MDP 7 47.77662340054132 30
average reward exploit: MDP 7 47.95521880585566 30
average reward exploit: MDP 7 47.690819225597146 30
average reward exploit: MDP 7 47.9010055369964 30
average reward explore: 58.706389927482164
average reward exploit: MDP 8 46.26242564336824 30
average reward exploit: MDP 8 45.49098012674761 30
average reward exploit: MDP 8 45.85799870462668 30
average reward exploit: MDP 8 46.226214028199

In [9]:
explore_paths, explore_rewards = a.sample_paths_explore(a.env,1)

In [10]:
print("average reward explore:", np.mean(explore_rewards))
M = a.stack_trajectories(explore_paths)

average reward explore: 78.12981287064275


In [11]:
Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })

In [12]:
Z = np.reshape(Z,[1,len(Z)])

In [13]:
exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1)

In [14]:
print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))

average reward exploit 4466.792213821265 2000


In [21]:
a.num_traj = 2000

In [ ]:
for i in range(20):
    explore_paths, explore_rewards = a.sample_paths_explore(a.env,30*np.random.rand())
    print("gravity:", a.sim.model.opt.gravity)
    print("average reward explore:", np.mean(explore_rewards))
    M = a.stack_trajectories(explore_paths)
    Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
    Z = np.reshape(Z,[1,len(Z)])
    exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1)
    print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))
    

gravity: [  0.           0.         -14.23799705]
average reward explore: 55.876055596810076
average reward exploit 4131.248029732218 2000
gravity: [  0.           0.         -28.92469711]
average reward explore: 26.375929544684027
average reward exploit 4128.3822263169 2000
gravity: [  0.           0.         -13.78489803]
average reward explore: 56.98693803520648
average reward exploit 4131.243695054299 2000
gravity: [  0.           0.         -13.99769514]
average reward explore: 56.371840402102315
average reward exploit 4127.052310839154 2000
gravity: [  0.           0.         -24.81371372]
average reward explore: 35.30427806908966
average reward exploit 4130.655895877959 2000
gravity: [  0.          0.        -20.5110446]
average reward explore: 39.77185976141281
average reward exploit 4126.369389991184 2000
gravity: [  0.           0.         -24.32955676]
average reward explore: 36.2316832686866
average reward exploit 4124.35278186741 2000
gravity: [  0.           0.         -1

In [ ]:
observations_explore = np.concatenate([path["observation"] for path in explore_paths])
new_observations_explore = np.concatenate([path["new_obs"] for path in explore_paths])
new_actions_explore = np.concatenate([path["new_acs"] for path in explore_paths])
actions_explore = np.concatenate([path["action"] for path in explore_paths])
rewards_explore = np.concatenate([path["reward"] for path in explore_paths])
returns_explore = a.get_returns(explore_paths, explore = True)

In [ ]:
M = np.array(a.stack_trajectories(explore_paths))
Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
Z = np.reshape(Z,[1,len(Z)])

In [ ]:
 print("average reward explore: MDP", np.sum(explore_rewards)/num_traj, len(explore_rewards))

In [ ]:
exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1000)

In [ ]:
print("average reward exploit: MDP", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))

In [ ]:
def randomize_pendulum(env):
    m = config['pendulum_mass_min'] + np.random.rand()*(config['pendulum_mass_max'] - config['pendulum_mass_min'])
    l = config['pendulum_len_min'] + np.random.rand()*(config['pendulum_len_max'] - config['pendulum_len_min'])
    env.m = m
    env.l = l

In [ ]:
def randomize_hopper(env):
    ts = config['torso_min'] + np.random.rand()*(config['torso_max'] - config['torso_min'])
    f = config['friction_min'] + np.random.rand()*(config['friction_max'] - config['friction_min'])
    
    env.friction = f
    env.torso_size = ts
    env.apply_env_modifications()

In [ ]:
import yaml
cfg_filename = 'hopper-config.yml'
with open(cfg_filename,'r') as ymlfile:
    config = yaml.load(ymlfile)

In [ ]:
env = Randomizer(gym.make('Hopper-v2'), randomize_hopper)
env.unwrapped.__dict__.keys()
env.reset()

In [ ]:
m = config['pendulum_mass_min'] + np.random.rand()*(config['pendulum_mass_max'] - config['pendulum_mass_min'])

In [ ]:
config

In [ ]:
env_id = 'Hopper-v2'
e = gym.make(env_id)

In [ ]:
e.unwrapped.__dict__.keys()

In [ ]:
e.unwrapped.apply_env_modifications()

In [7]:
a.sim.model.opt